This notebook goes through the following with TensorFlow:

- build a transfer learning feature extraction model using TensorFlow Hub
- use TensorBoard callback to track and compare model training results

In [ ]:
# Add timestamp
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")

Notebook last run (end-to-end): 2024-01-04 18:06:01.153224


In [ ]:
# Check if we're using GPU
!nvidia-smi

Thu Jan  4 18:06:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## download data (10% of the food vision data)

In [ ]:
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
# Unzip data
zip_data = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_data.extractall()
zip_data.close()

--2024-01-04 18:06:01--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.161.207, 74.125.126.207, 74.125.132.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.161.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.2’

10_food_classes_10_ 100%[===================>] 160.74M  97.2MB/s    in 1.7s    

2024-01-04 18:06:03 (97.2 MB/s) - ‘10_food_classes_10_percent.zip.2’ saved [168546183/168546183]



## Check data

In [ ]:
import os

for root, dirs, files in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirs)} directories and {len(files)} images under '{root}'.")

There are 2 directories and 0 images under '10_food_classes_10_percent'.
There are 10 directories and 0 images under '10_food_classes_10_percent/train'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images under '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images under 

## Normalization and create data batches

In [ ]:
# Normalization and create data batches
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"
batch_size = 32
img_size = (224, 224)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=img_size,
                                               batch_size=batch_size,
                                               class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=img_size,
                                             batch_size=batch_size,
                                             class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


## Set up TensorBoard callbacks

[Callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks) are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks include:

- Experiment tracking with TensorBoard - **log the performance** of multiple models and then view and compare these models in a visual way on TensorBoard. Helpful to compare the results of different models on your data.
- Model checkpointing - save your model as it trains so you can stop training if needed and come back to continue off where you left. Helpful if training takes a long time and can't be done in one sitting.
- Early stopping - leave your model training for an arbitrary amount of time and have it stop training automatically when it ceases to improve. Helpful when you've got a large dataset and don't know how long training will take.




### Create a TensorBoard callback

We want to save a model's performance logs to a directory named [dir_name]/[experiment_name]/[current_timestamp], where:

- dir_name is the overall logs directory
- experiment_name is the particular experiment
- current_timestamp is the time the experiment started based on Python's datetime.datetime().now()

In [ ]:
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  return tensorboard_callback

## Creating models using TensorFlow Hub

> 🤔 **Question:** *I thought we were doing image classification, why do we choose feature vector and not classification?*

Differnet types of transfer learning come into play, as is, feature extraction and fine-tuning:

- **"As is" transfer learning** is when you take a pretrained model as it is and **apply it to your task without any changes**.

  * Model's with `"/classification"` in their name on TensorFlow Hub provide this kind of functionality.

- **Feature extraction transfer learning**: take the weights a pretrained model has learned and **adjust its outputs** to be more suited to your problem.

  * For example, say the pretrained model you were using had 236 different layers (EfficientNetB0 has 236 layers), but the top layer outputs 1000 classes because it was pretrained on ImageNet. To adjust this to your own problem, you might remove the original activation layer and replace it with your own but with the right number of output classes. The important part here is that **only the top (last) few layers become trainable, the rest remain frozen**.

- **Fine-tuning transfer learning**:  take the weights and **adjust (fine-tune) them** to your own problem.

    * This usually means training **some, many or all** of the layers in the pretrained model. This is useful when you've got a large dataset (e.g. 100+ images per class) where your data is slightly different to the data the original model was trained on.


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [ ]:
resnet_url = "https://kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-feature-vector/versions/1"
efficientnet_url = "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b0-feature-vector/versions/2"

In [ ]:
img_size+(3,)

(224, 224, 3)

In [ ]:
def create_model_with_feature_extraction(weights_url, num_classes=10):
  """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """
  feature_extract_layer = hub.KerasLayer(weights_url,
                                         trainable=False,
                                         input_shape=img_size+(3,),
                                         name="feature_extraction_layer")

  model = tf.keras.Sequential([feature_extract_layer,
                               layers.Dense(num_classes,
                                            name="output_layer",
                                            activation="softmax")])

  return model

In [ ]:
resnet_model = create_model_with_feature_extraction(resnet_url,
                                                    train_data.num_classes)

resnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=['accuracy']
)

resnet_history=resnet_model.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=create_tensorboard_callback(
        dir_name='tensorflow_hub',
        experiment_name='resnet_v2'
    )
)

Epoch 1/5
24/24 [==============================] - 23s 660ms/step - loss: 2.0265 - accuracy: 0.3360 - val_loss: 1.3159 - val_accuracy: 0.5844
Epoch 2/5
24/24 [==============================] - 10s 443ms/step - loss: 0.9218 - accuracy: 0.7173 - val_loss: 0.8718 - val_accuracy: 0.7244
Epoch 3/5
24/24 [==============================] - 11s 472ms/step - loss: 0.6344 - accuracy: 0.8067 - val_loss: 0.7568 - val_accuracy: 0.7676
Epoch 4/5
24/24 [==============================] - 11s 468ms/step - loss: 0.4711 - accuracy: 0.8760 - val_loss: 0.7037 - val_accuracy: 0.7752
Epoch 5/5
24/24 [==============================] - 11s 483ms/step - loss: 0.3808 - accuracy: 0.9133 - val_loss: 0.6787 - val_accuracy: 0.7856


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 2048)              23564800  
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23585290 (89.97 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [ ]:
efficientnet_model = create_model_with_feature_extraction(efficientnet_url,
                                                          train_data.num_classes)

efficientnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=['accuracy']
)

efficientnet_history = efficientnet_model.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=create_tensorboard_callback(
        dir_name="tensorflow_hub",
        experiment_name="efficientnet_v2"
    )
)

Epoch 1/5
24/24 [==============================] - 22s 561ms/step - loss: 2.0171 - accuracy: 0.3387 - val_loss: 1.5481 - val_accuracy: 0.5996
Epoch 2/5
24/24 [==============================] - 10s 436ms/step - loss: 1.3198 - accuracy: 0.6933 - val_loss: 1.1376 - val_accuracy: 0.7064
Epoch 3/5
24/24 [==============================] - 10s 434ms/step - loss: 1.0114 - accuracy: 0.7507 - val_loss: 0.9527 - val_accuracy: 0.7428
Epoch 4/5
24/24 [==============================] - 9s 401ms/step - loss: 0.8417 - accuracy: 0.8067 - val_loss: 0.8599 - val_accuracy: 0.7572
Epoch 5/5
24/24 [==============================] - 10s 433ms/step - loss: 0.7347 - accuracy: 0.8333 - val_loss: 0.8002 - val_accuracy: 0.7672


In [ ]:
efficientnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 1280)              5919312   
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 5932122 (22.63 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 5919312 (22.58 MB)
_________________________________________________________________
